In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from pyrsgis import raster

In [ ]:
builtup = 'dades/r_20_09_2017.tif'
# Import pre-trained model
nn = keras.models.load_model("dades/nn_fire_risk.h5")
# Import means and deviations from training dataset to standarize
mean = pd.read_csv("data/means.csv", index_col=0, header=0, squeeze=True)
std = pd.read_csv("data/stds.csv", index_col=0, header=0, squeeze=True)

In [ ]:
# Read raster as array
ds, features = raster.read(builtup, bands='all')
features = features.transpose(1,2,0).reshape((features.shape[1]*features.shape[2]), features.shape[0])

In [ ]:
# Import into pandas DataFrame
df = pd.DataFrame(features)

In [ ]:
# Select quantitatives variables
quant_var = df.iloc[:, df.columns != 8]

In [ ]:
# Normalize
normalized_df = (quant_var-mean.values)/std.values
# Add soil use binnarized
normalized_df = normalized_df.join(pd.get_dummies(df[8].astype("category")), rsuffix="dummy")

In [ ]:
# Predict new data
predicted = nn.predict(normalized_df)

In [ ]:
# Export probability raster    
prediction = np.reshape(list(predicted[:,1]), (ds.RasterYSize, ds.RasterXSize))
outFile = 'dades/prediction_20_09_2017.tif'
raster.export(prediction, ds, filename=outFile, dtype='float')